# Air Pollution Prediction

The goal of this project is to predict the pollution of air, specifically the concentration of PM2.5 in the air. We use two models: a Lasso Regressor and a Random Forest Regressor and compare their performances.

### Data

For the purpose of the project, we are going to use a dataset of air pollution measurements in Beijing archived by the UCI. To read about the dataset visit the [UCI page](https://archive.ics.uci.edu/dataset/501/beijing+multi+site+air+quality+data).

Over the course of the project, we are going to:
  1. Preprocess the data
  2. Build our own Lasso-regression
  3. Use sklearn's tools to perform regression on the data
  
Let's start by importing the libraries we will be using.

In [263]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

Download the data from the following link and save it to a CSV file named 'data.csv'.

In [264]:
urllib.request.urlretrieve('https://drive.google.com/uc?id=1m1g4Xn1wMAGV_EU0Nh1HTI1ogA3-tqJk&export=download', './data.csv')

('./data.csv', <http.client.HTTPMessage at 0x7ce35d8cd240>)

In [265]:
df = pd.read_csv('data.csv')
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,9.0,9.0,6.0,17.0,200.0,62.0,0.3,1021.9,-19.0,0.0,WNW,2.0,Wanshouxigong
1,2,2013,3,1,1,11.0,11.0,7.0,14.0,200.0,66.0,-0.1,1022.4,-19.3,0.0,WNW,4.4,Wanshouxigong
2,3,2013,3,1,2,8.0,8.0,NaN,16.0,200.0,59.0,-0.6,1022.6,-19.7,0.0,WNW,4.7,Wanshouxigong
3,4,2013,3,1,3,8.0,8.0,3.0,16.0,NaN,NaN,-0.7,1023.5,-20.9,0.0,NW,2.6,Wanshouxigong
4,5,2013,3,1,4,8.0,8.0,3.0,NaN,300.0,36.0,-0.9,1024.1,-21.7,0.0,WNW,2.5,Wanshouxigong


In [266]:
df.set_index(['No'], inplace=True) #set No column as the index column

We will be trying to predict the pollution (PM2.5) using:
- temperature ('TEMP')
- pressure ('PRES')
- dew-point temperature ('DEWP')
- precipitation ('RAIN')
- wind direction ('wd')
- wind speed ('WSPM')

But frist we need to clean the data.

## Data Cleaning

In [267]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35064 entries, 1 to 35064
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     35064 non-null  int64  
 1   month    35064 non-null  int64  
 2   day      35064 non-null  int64  
 3   hour     35064 non-null  int64  
 4   PM2.5    34368 non-null  float64
 5   PM10     34580 non-null  float64
 6   SO2      34395 non-null  float64
 7   NO2      34310 non-null  float64
 8   CO       33767 non-null  float64
 9   O3       33986 non-null  float64
 10  TEMP     35045 non-null  float64
 11  PRES     35045 non-null  float64
 12  DEWP     35045 non-null  float64
 13  RAIN     35045 non-null  float64
 14  wd       34985 non-null  object 
 15  WSPM     35051 non-null  float64
 16  station  35064 non-null  object 
dtypes: float64(11), int64(4), object(2)
memory usage: 4.8+ MB


In [268]:
# drop the null rows in our columns of interest
df = df.drop(columns = ['year','month','day','PM10','SO2','NO2','CO','O3','station'], axis=1)
df = df.dropna(subset=['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM'])
# reorder the columns so the column to be predicted PM2.5 is the fir column
df = df[['PM2.5','hour','TEMP','PRES','DEWP','RAIN','wd','WSPM']]
df.isnull().sum()

PM2.5    0
hour     0
TEMP     0
PRES     0
DEWP     0
RAIN     0
wd       0
WSPM     0
dtype: int64

## Train-test Split

We perform the split before preprocessing so the test data can be left unprocessed in order for it to match real world scenarios when the model is deployed. We will use 85% for the training and validation data, and 15% for the test set. To avoid unwanted correlations, we use split the data according to time. Thankfully the data comes ordered by time so we do not need to order it by ourselves.

In [269]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.15, shuffle=False)

len(train_df)/len(df),len(test_df)/len(df) #checking the sizes manually

(0.8499883327499709, 0.15001166725002918)

## Lasso Regression

We will now manually implemenet Lasso Regression from scratch.

[From Wikipedia](https://en.wikipedia.org/wiki/Lasso_(statistics)#:~:text=In%20statistics%20and%20machine%20learning,of%20the%20resulting%20statistical%20model.): "Lasso (least absolute shrinkage and selection operator) is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the resulting statistical model."

### OLS Regression

First we perform OLS regression and we will later regularize it using Lasso regression. Begin by defining the prediciton fuction for linear regression i.e. $f(X,\mathbf{w}) = X \mathbf{w}$.

In [270]:
# Prediction fuction for linear regression
def prediction(X,w):
    return X@w

Next, the objective function to tell us the error, mathematically given as: $E = \sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2$.

In [271]:
# Objective function to return the error
def objective(X,y,w):
    """
    Computes the sum squared error (for us to perform OLS linear regression).
    """
    f = prediction(X,w)
    return np.sum((y-f)**2)

Now we find the derivative of the objective w.r.t. the parameter vector. The gradient of the error function for linear regression is:

$\frac{\partial E}{\partial \mathbf{w}} = 2 X^\top X \mathbf{w} - 2 X^\top y$.

In [272]:
# gradient of the error function
def objective_derivative(X,y,w):
    """
    Computes the derivative of the sum squared error, wrt the parameters.
    """
    return 2*X.T@X@w-2*X.T@y

To check the gradients are correct, we can estimate the gradient numerically:

In [273]:
def numerical_objective_derivative(X,y,w):
    """
    Computes a numerical approximation to the derivative of the sum squared error, wrt the parameters.
    """
    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective(X,y,w+d)-objective(X,y,w-d))/2e-6
    return g

We'll use a toy design matrix & labels to use to check our code works. We'll also specify a weight vector too, for testing. The two gradient vectors should be approximately equal:

In [274]:
X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([0.0,1,2])
w = np.array([1.0,2])

objective_derivative(X,y,w),numerical_objective_derivative(X,y,w)

(array([39.28, 73.2 ]), array([39.28, 73.2 ]))

Now, we ptimise $\mathbf{w}$ to minimise the objective. We will use the gradient function we wrote to maximise $\mathbf{w}$ using gradient descent. At each iteration: compute the gradient and subtract the scaled gradient from the w parameter.

In [275]:
from numpy.lib.function_base import gradient
def optimise_parameters(X,y,startw):
    """
    Returns the w that minimises the objective.
    """
    w = startw
    for i in range(1000):
      scaled_gradient = objective_derivative(X,y,startw) * 0.001 #0.01 is the learning rate
      w -= scaled_gradient
    return w

Let's compute our answers and compare it to the answer provided by sklearn:

In [276]:
bestw = optimise_parameters(X,y,w)
print(bestw)

[0.60568751 0.1808409 ]


In [277]:
from sklearn import linear_model
clf = linear_model.LinearRegression(fit_intercept=False)
clf.fit(X,y)
print(clf.coef_) #matches the value of w we found above, hopefully!

[0.83333333 0.05555556]


### Regularization

We can now regularize our regression function above using Lasso Regression.

We need to include the Lasso regularization term to our objective function. This gives us a **a new objective function:**

$$E = \frac{1}{2N}\sum_{i=1}^N \big(y_i-f(\mathbf{x}_i,\mathbf{w}) \big)^2 + \alpha \sum_{j=1}^P |w_j|$$

In [278]:
def objective_lasso(X,y,w,alpha):
    """
    Computes half the mean squared error, with an additional L1 regularising term. alpha controls the level of regularisation.
    """
    return objective(X,y,w)/(2*y.shape[0])+ alpha*np.sum(abs(w))

Similar to before, we will be computing the gradient. The derivative of the objective function is then given as:

$$\frac{\partial E}{\partial \mathbf{w}} = \frac{1}{N}(X^\top X \mathbf{w} - X^\top y) + \alpha\;\text{sign}(\mathbf{w}).$$

In [279]:
def objective_lasso_derivative(X,y,w,alpha):
    """
    Returns the derivative of the Lasso objective function.
    """
    return (X.T@X@w-X.T@y)/y.shape[0] + alpha*np.sign(w)

We can check it again, numerically. The two pairs of parameters should be the same:

In [280]:
def numerical_objective_lasso_derivative(X,y,w,alpha):
    """
    This finds a numerical approximation to the true gradient
    """

    g = np.zeros_like(w)
    for i,wi in enumerate(w):
        d = np.zeros_like(w)
        d[i]=1e-6
        g[i] = (objective_lasso(X,y,w+d,alpha)-objective_lasso(X,y,w-d,alpha))/2e-6
    return g

objective_lasso_derivative(X,y,w,0.1),numerical_objective_lasso_derivative(X,y,w,0.1)

(array([0.05776258, 0.12324545]), array([0.05776258, 0.12324545]))

Lastly, optimise $\mathbf{w}$ to minimise the Lasso objective.

In [328]:
def optimise_parameters_lasso(X,y,startw):
    """
    Returns the w that minimises the Lasso objective.
    """
    alpha = 0.1
    for i in range(5000):
      scaled_gradient_q10 = objective_lasso_derivative(X,y,startw,alpha) * 0.01
      startw -= scaled_gradient_q10
    return startw

X = np.array([[0.0,0],[1,3],[2.2,3]])
y = np.array([0.0,1,2])
w = np.array([1.0,2])
optimise_parameters_lasso(X,y,w)

array([0.63884183, 0.14261849])

We can check against the sklearn method:

In [179]:
clf = linear_model.Lasso(alpha=0.1,fit_intercept=False)
clf.fit(X,y)
print(clf.coef_)

[0.63931263 0.1423666 ]


## Data Preprocessing
### One hot Encoding
The wind direction column 'wd' is a string type and must be converted to a numerical column. This is achieved by applying one-hot encoding.


In [180]:
def add_wd_onehot(df):
    """Add new one-hot encoding set of columns, removes the old column it's made from. Returns new dataframe."""
    # Get one hot encoding of column 'wd'
    onehot_df = pd.get_dummies(df.wd, prefix='wd')
    # Drop column as it is now encoded
    df = df.drop(columns=['wd'], axis=1)
    # Join the encoded df
    onehot_df = df.join(onehot_df)
    return onehot_df

train_df_wdencoded = add_wd_onehot(train_df)
train_df_wdencoded

,PM2.5,hour,TEMP,PRES,DEWP,RAIN,WSPM,wd_E,wd_ENE,wd_ESE,...,wd_NNW,wd_NW,wd_S,wd_SE,wd_SSE,wd_SSW,wd_SW,wd_W,wd_WNW,wd_WSW
No,,,,,,,,,,,,,,,,,,,,,
1,9.0,0,0.3,1021.9,-19.0,0.0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,11.0,1,-0.1,1022.4,-19.3,0.0,4.4,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,8.0,2,-0.6,1022.6,-19.7,0.0,4.7,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,8.0,3,-0.7,1023.5,-20.9,0.0,2.6,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,8.0,4,-0.9,1024.1,-21.7,0.0,2.5,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29760,138.0,23,28.2,997.0,24.2,0.0,1.0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29761,145.0,0,27.7,997.1,24.3,0.0,0.9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
29762,139.0,1,26.9,996.6,24.6,0.0,0.8,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Data Standardization

Make the mean zero and the standard deviation one.

In [181]:
def standardise(df):
    """
    Returns a new dataframe in which all but the PM2.5 columns are standardised (i.e. have a mean of zero and standard deviation of 1).
    """
    for col in df.iloc[:,1:].columns:
      df[col] = (df[col]-df[col].mean())/df[col].std()
    df_standardized = df.iloc[:,:1].join(df.iloc[:,1:])
    return df_standardized

train_df_preprocessed = standardise(add_wd_onehot(train_df))
test_df_preprocessed = standardise(add_wd_onehot(test_df))

Seperate the features (X) from the output (y) and put them into arrays.

In [182]:
X_train = train_df_preprocessed.iloc[:,1:].to_numpy()
y_train = train_df_preprocessed.iloc[:,0].to_numpy()

X_test = test_df_preprocessed.iloc[:,1:].to_numpy()
y_test = test_df_preprocessed.iloc[:,0].to_numpy()

In [183]:
X_train.shape[1]

22

## Fitting the Lasso Model and Model Evaluation (via RMSE)

Let evaluate the performance of the sklearn classifier using RMSE.

In [336]:
#tain sklearn's lasso classifier
clf = linear_model.Lasso(alpha=0.1,fit_intercept=True) #we'll turn fit_intercept on, as we've not added a '1's column to our design matrix.
clf.fit(X_train,y_train)

#predicted and evaluate
rmse_sklearn_train = np.sqrt(np.mean((clf.predict(X_train)-y_train)**2))
rmse_sklearn_test = np.sqrt(np.mean((clf.predict(X_test)-y_test)**2))

rmse_sklearn_train,rmse_sklearn_test

(72.40451940025356, 88.63619480960364)

The performance can be much better.

## Random Forest

To get better predictions, we will used a random forest regressor as a linear model is not enought to capture our data.

To find the best parameters for making predictions, we will use a cross-validated grid search.



The final step is to use a random forest regressor.

If we use the default random forest regressor, we find we get considerable over-fitting. So we need to explore different parameters. We will use a cross-validated grid search over the parameters:

- max_features: The number of features to consider when looking for the best split (i.e. controls subsampling), *from 1 to the number of features* in 4 steps (e.g. use `np.linspace`)
- n_estimators: The number of trees in the forest, from 10 to 100 in 4 steps.
- max_samples : the number of samples to draw from to train each base estimator, from 0.1 to 0.9 in 4 steps.

We will use `GridSearchCV`.

Have a look at the documentation for this, the three parameters we need to specify are:

- the 'estimator': an **INSTANCE** of RandomForestRegressor.
- param_grid: a **DICTIONARY**, each item is the title of the parameter, and equals an array of the values we need to test. For exmaple, one of the items might be `{'max_samples': np.linspace(0.1, 0.9, 5)}`.
- You'll need to think carefully how to make the lists for the `max_features` and `n_estimators` as these both need to be (positive) integers. E.g. use `.astype(int)`.

In [332]:
#1. Create a grid of parameter values for n_estimators, max_features and max_samples,
n_estimators = np.linspace(10, 100, 4).astype(int)
max_features = np.linspace(1, X_train.shape[1], 4).astype(int)
max_samples = np.linspace(0.1, 0.9, 4)
param_grid = dict(n_estimators=n_estimators,max_features=max_features,max_samples=max_samples)
#2. Create a GridSearchCV object, using the random forest regressor:
grid_regression = GridSearchCV(RandomForestRegressor(), param_grid=param_grid)
#Note: Because there is so much training data, using the full dataset takes too long. So here we'll just use 10%
np.random.seed(42)
idx = np.sort(np.random.choice(len(X_train), size=int(len(X_train)*0.1), replace=False))
#3. Fit to training data in (the subset of) X_train and y_train
grid_regression.fit(X_train[idx,:],y_train[idx])

GridSearchCV(estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  8, 15, 22]),
                         'max_samples': array([0.1       , 0.36666667, 0.63333333, 0.9       ]),
                         'n_estimators': array([ 10,  40,  70, 100])})

Here we print the best parameters from the grid search (on the training/validation cross-validation run):

In [333]:
best_n_estimators = grid_regression.best_params_['n_estimators']
best_max_features = grid_regression.best_params_['max_features']
best_max_samples = grid_regression.best_params_['max_samples']
print('Best n_estimator', best_n_estimators)
print('Best max features', best_max_features)
print('Best max samples', best_max_samples)

Best n_estimator 40
Best max features 8
Best max samples 0.6333333333333333


Finally compute the RMSE for the training and test data:

In [334]:
#Put answer here
rmse_rf_train = np.sqrt(np.mean((grid_regression.predict(X_train)-y_train)**2))
rmse_rf_test = np.sqrt(np.mean((grid_regression.predict(X_test)-y_test)**2))

rmse_rf_train,rmse_rf_test

(64.08343121285556, 85.98106311956076)

We can compare this to the standard deviations for the two sets of data.

In [335]:
np.std(y_train), np.std(y_test)

(82.43136001277581, 103.25109103178704)

### Did the random forest do better than lasso regression?

Yes, Random Forest performed better than the Lasso Regressor because Random Forest had a lower RMSE for both training and testing sets: `(64.08343121285556, 85.98106311956076)`, respectively, than the Lasso Regressor: `(72.40451940025356, 88.63619480960364)`.